cliptext 1st embedding:\
corr: `0.9999999999999798` euclidian dist: `0.08716399866359316`\
clipvision 1st embedding:\
corr: `0.7206470311671278` euclidian dist: `0.7445045003979572`\
autokl:\
corr: `0.021463346119866764` euclidian dist: `113.58405093418304`\
\
THINGS-MEG unmodified regression performance\
cliptext 1st embedding: \
corr: `0.7132301973694442` euclidian dist: `0.6739413756833288` \
clipvision 1st embedding: \
corr: `0.6127160358947972` euclidian dist: `0.878658145980226` \
autokl:\
corr: `0.005573586983020162` euclidian dist: `114.27715624831653`

In [12]:
from models import SimpleConv, BrainModule
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch
from torch import nn, optim
import numpy as np
from collections import namedtuple
from tqdm import tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt
from scipy import stats
from scipy.spatial.distance import correlation

In [13]:
data_array = np.load('cache/processed_data/BIGMEG1/train_thingsmeg_sub-BIGMEG1.npy', mmap_mode='r')
labels_array = np.load('cache/extracted_embeddings/BIGMEG1/train_cliptext1b_sub-BIGMEG1.npy', mmap_mode='r')
# labels_array = np.load('cache/extracted_embeddings/BIGMEG1/train_clipvision1b_sub-BIGMEG1.npy', mmap_mode='r')
labels_array = labels_array[:, 0]
data_array.shape, labels_array.shape

((22248, 272, 120), (22248, 768))

In [14]:
device = torch.device('cuda:7')
model = BrainModule()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
model.to(device);

/home/AD/tfei/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# Optional: see network parameters
# for name, param in model.named_parameters():
#     print(f"Layer: {name} | Number of parameters: {param.numel()}")
# total_params = sum(p.numel() for p in model.parameters())
# print(f"Total number of parameters: {total_params}")

In [16]:
data_tensor = torch.tensor(data_array).float()
labels_tensor = torch.tensor(labels_array).float()
# Create a TensorDataset from your data tensor and labels tensor
dataset = TensorDataset(data_tensor, labels_tensor)

train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size  # 20% for validation
train_data, val_data = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_data, batch_size=32)
val_dataloader = DataLoader(val_data, batch_size=32)

# Training loop
for epoch in range(150):  # 100 epochs
    # Training phase
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation
        val_loss = 0
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
        val_loss /= len(val_dataloader)
    model.train()  # Set the model back to training mode

    clear_output(wait=False)
    print(f'Epoch {epoch+1} / {150}, Training Loss: {loss.item()}, Validation Loss: {val_loss}')

Epoch 150 / 150, Training Loss: 0.0002909325703512877, Validation Loss: 0.0003247078474877136


In [17]:
test_data = np.load('cache/processed_data/BIGMEG1/test_thingsmeg_sub-BIGMEG1.npy', mmap_mode='r')
test_labels = np.load('cache/extracted_embeddings/BIGMEG1/test_cliptext1b_sub-BIGMEG1.npy', mmap_mode='r')
# test_labels = np.load('cache/extracted_embeddings/BIGMEG1/test_clipvision1b_sub-BIGMEG1.npy', mmap_mode='r')
test_labels = test_labels[:, 0]

test_data_tensor = torch.tensor(test_data).float()
test_labels_tensor = torch.tensor(test_labels).float()

test_dataset = TensorDataset(test_data_tensor, test_labels_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Testing loop
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    preds = []
    for inputs, labels in test_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds.extend(outputs.cpu().numpy())
    pred_labels = np.array(preds)
# Compute the Euclidean and correlation distances
euclidean_distances = np.array([np.linalg.norm(u - v) for u, v in zip(pred_labels, test_labels)])
correlation_distances = np.array([correlation(u, v) for u, v in zip(pred_labels, test_labels)])
# Compute the average Euclidean distance
average_euclidean_distance = euclidean_distances.mean()
correlations = (1 - correlation_distances).mean()
print('corr:', correlations, 'euclidian dist:' ,average_euclidean_distance)

corr: 0.8331517203063764 euclidian dist: 0.4900515302499065


In [ ]:
# # save pred cliptext
# preds_array = np.array(preds)
# preds_repeated = np.repeat(preds_array[:, np.newaxis, :], 77, axis=1)
# # np.save('cache/predicted_embeddings/BIGMEG1/thingsmeg_dummymodule_cliptext1b_sub-BIGMEG1.npy', preds_repeated)

# # save pred clipvision
# preds_array = np.array(preds)
# regress_labels = np.load('cache/predicted_embeddings/BIGMEG1/thingsmeg_regress_clipvision1b_sub-BIGMEG1.npy')
# regress_labels[:, 0] = preds_array
# # np.save('cache/predicted_embeddings/BIGMEG1/thingsmeg_dummyregress_clipvision1b_sub-BIGMEG1.npy', regress_labels)

## reference metrics

In [10]:
# NSD fMRI regression performance
test_labels = np.load('/mnt/sphere/projects/brain-diffuser/data/extracted_features/subj01/nsd_cliptext_test.npy', mmap_mode='r')
test_labels = test_labels[:, 0, :]
pred_labels = np.load('/mnt/sphere/projects/brain-diffuser/data/predicted_features/subj01/nsd_cliptext_predtest_nsdgeneral.npy', mmap_mode='r')
pred_labels = pred_labels[:, 0, :]
# Compute the Euclidean distances
euclidean_distances = np.array([np.linalg.norm(u - v) for u, v in zip(pred_labels, test_labels)])
correlation_distances = np.array([correlation(u, v) for u, v in zip(pred_labels, test_labels)])
# Compute the average Euclidean distance
average_euclidean_distance = euclidean_distances.mean()
correlations = (1 - correlation_distances).mean()
print('cliptext 1st embedding:')
print('corr:', correlations, 'euclidian dist:', average_euclidean_distance)
test_labels = np.load('/mnt/sphere/projects/brain-diffuser/data/extracted_features/subj01/nsd_clipvision_test.npy', mmap_mode='r')
test_labels = test_labels[:, 0, :]
pred_labels = np.load('/mnt/sphere/projects/brain-diffuser/data/predicted_features/subj01/nsd_clipvision_predtest_nsdgeneral.npy', mmap_mode='r')
pred_labels = pred_labels[:, 0, :]
# Compute the Euclidean distances
euclidean_distances = np.array([np.linalg.norm(u - v) for u, v in zip(pred_labels, test_labels)])
correlation_distances = np.array([correlation(u, v) for u, v in zip(pred_labels, test_labels)])
# Compute the average Euclidean distance
average_euclidean_distance = euclidean_distances.mean()
correlations = (1 - correlation_distances).mean()
print('clipvision 1st embedding:')
print('corr:', correlations, 'euclidian dist:', average_euclidean_distance)
test_labels = np.load('/mnt/sphere/projects/brain-diffuser/data/extracted_features/subj01/nsd_vdvae_features_31l.npz', mmap_mode='r')
test_labels = test_labels['test_latents']
pred_labels = np.load('/mnt/sphere/projects/brain-diffuser/data/predicted_features/subj01/nsd_vdvae_nsdgeneral_pred_sub1_31l_alpha50k.npy', mmap_mode='r')
# Compute the Euclidean distances
euclidean_distances = np.array([np.linalg.norm(u - v) for u, v in zip(pred_labels, test_labels)])
correlation_distances = np.array([correlation(u, v) for u, v in zip(pred_labels, test_labels)])
# Compute the average Euclidean distance
average_euclidean_distance = euclidean_distances.mean()
correlations = (1 - correlation_distances).mean()
print('autokl:')
print('corr:', correlations, 'euclidian dist:', average_euclidean_distance)

cliptext 1st embedding:
corr: 0.9999999999999798 euclidian dist: 0.08716399866359316
clipvision 1st embedding:
corr: 0.7206470311671278 euclidian dist: 0.7445045003979572
autokl:
corr: 0.021463346119866764 euclidian dist: 113.58405093418304


In [11]:
# THINGS-MEG unmodified regression performance
test_labels = np.load('cache/extracted_embeddings/BIGMEG1/test_cliptext1b_sub-BIGMEG1.npy', mmap_mode='r')
test_labels = test_labels[:, 0, :]
pred_labels = np.load('cache/predicted_embeddings/BIGMEG1/thingsmeg_regress_cliptext1b_sub-BIGMEG1.npy', mmap_mode='r')
pred_labels = pred_labels[:, 0, :]
# Compute the Euclidean distances
euclidean_distances = np.array([np.linalg.norm(u - v) for u, v in zip(pred_labels, test_labels)])
correlation_distances = np.array([correlation(u, v) for u, v in zip(pred_labels, test_labels)])
# Compute the average Euclidean distance
average_euclidean_distance = euclidean_distances.mean()
correlations = (1 - correlation_distances).mean()
print('cliptext 1st embedding:')
print(correlations, average_euclidean_distance)
test_labels = np.load('cache/extracted_embeddings/BIGMEG1/test_clipvision1b_sub-BIGMEG1.npy', mmap_mode='r')
test_labels = test_labels[:, 0, :]
pred_labels = np.load('cache/predicted_embeddings/BIGMEG1/thingsmeg_regress_clipvision1b_sub-BIGMEG1.npy', mmap_mode='r')
pred_labels = pred_labels[:, 0, :]
# Compute the Euclidean distances
euclidean_distances = np.array([np.linalg.norm(u - v) for u, v in zip(pred_labels, test_labels)])
correlation_distances = np.array([correlation(u, v) for u, v in zip(pred_labels, test_labels)])
# Compute the average Euclidean distance
average_euclidean_distance = euclidean_distances.mean()
correlations = (1 - correlation_distances).mean()
print('clipvision 1st embedding:')
print(correlations, average_euclidean_distance)
test_labels = np.load('cache/extracted_embeddings/BIGMEG1/test_autokl1b_sub-BIGMEG1.npy', mmap_mode='r')
pred_labels = np.load('cache/predicted_embeddings/BIGMEG1/thingsmeg_regress_autokl1b_sub-BIGMEG1.npy', mmap_mode='r')
# Compute the Euclidean distances
euclidean_distances = np.array([np.linalg.norm(u - v) for u, v in zip(pred_labels, test_labels)])
correlation_distances = np.array([correlation(u, v) for u, v in zip(pred_labels, test_labels)])
# Compute the average Euclidean distance
average_euclidean_distance = euclidean_distances.mean()
correlations = (1 - correlation_distances).mean()
print('autokl:')
print('corr:', correlations, 'euclidian dist:', average_euclidean_distance)

cliptext 1st embedding:
0.7132301973694442 0.6739413756833288
clipvision 1st embedding:
0.6127160358947972 0.878658145980226
autokl:
corr: 0.005573586983020162 euclidian dist: 114.27715624831653
